In [1]:
import os
import sys
import traceback
import pandas as pd
from llama_index.core import Document
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core.node_parser import SentenceSplitter
from dotenv import load_dotenv
from llama_index.llms.gemini import Gemini
import google.generativeai as genai



c:\Users\Tayyba\anaconda3\envs\my_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading the datasets

In [2]:


def preprocess_excel(file_path):
    # Read the Excel file
    df = pd.read_excel(file_path)
    
    # Clean column names
    df.columns = df.columns.str.strip().str.lower()
    
    # Create LlamaIndex documents with clear price focus
    documents = []
    for _, row in df.iterrows():
        product_name = row.get('product info', 'Unknown Product')
        price = row.get('price(rs)', 'Price not available')
        text = f"The price of {product_name} is {price}."
        documents.append(Document(text=text))
    
    return documents


documents = preprocess_excel(r"./data_set/data_file.xlsx")
documents

[Document(id_='ae45e6c7-dde1-4b9e-82ee-81472385085e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='The price of HMW-28100-28L Digital Grill Microwave Oven is 70999.', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='971dd821-72bc-46f1-b6ac-2e722aaac8f3', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='The price of HMW-23200-23L Digital Microwave Oven is 67449.', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 D

### Loading the API key (Gemini model) and the pretrained gemini models 

In [3]:
load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
try:
    model=Gemini(models='gemini-pro',api_key=GOOGLE_API_KEY)
except:
    traceback.print_exc()

In [5]:
try:
    gemini_embed_model = GeminiEmbedding(model_name="models/embedding-001")
    Settings.llm = model
    Settings.embed_model = gemini_embed_model
    Settings.node_parser = SentenceSplitter(chunk_size=256, chunk_overlap=20)
    index = VectorStoreIndex.from_documents(documents,service_context=Settings)
except:
    import traceback
    traceback.print_exc()
    

In [6]:
index.storage_context.persist()
query_engine = index.as_query_engine()


##### Generating the query response using the process of RAG system
##### if we give the better prompt, the model worked accordingly

In [9]:
lis = ['FFD Safety Device 5 Burner Hob (HCC-509DGG) price?', 'Tell me price of HW100-BP14929S3 - 10 kg Fully Automatic Front Load Washing Machine grey/20 ','HRF-622ICG 22CFT Side by Side Digital Inverter Refrigerator price','HRF-622ICG - 22CFT - Side by Side Digital Inverter Refrigerator price','HGL-20MXP7 - 20L Solo Microwave Oven price?','FFD Safety Device 5 Burner Hob (HCC-509DGG) price?', "color hc-12345 price", "HC-az123zza-tv price"]
for i in lis:
    response = query_engine.query(
                f"Provide precise price and detailed product information for: {i}. "
                f"If no specific information is available, clearly state that no matching product was found."
            )
    print(response)

The price of the FFD Safety Device 5 Burner Hob (HCC-509DGG) is 77000.

The price of the HW100-BP14929S3 - 10 kg Fully Automatic Front Load Washing Machine is 216999.

The price for the HRF-622ICG 22CFT Side by Side Digital Inverter Refrigerator is 321999.

The price for the HRF-622ICG - 22CFT - Side by Side Digital Inverter Refrigerator is 321999.

The price of the HGL-20MXP7 - 20L Solo Microwave Oven is 54599.

The price of the FFD Safety Device 5 Burner Hob (HCC-509DGG) is 77000.

No matching product was found.

No matching product was found.

